In [1]:
# import hashlib
import hashlib

# Using hashlib.shake_256() method
gfg = hashlib.shake_256()
gfg.update(b'\x12\x34')

print(gfg.hexdigest(64))

d003ba6b7919a4384003f645d4a25c7ee631d249edebfbdf99eeb5ee31c997edcc19a439932e29389c7be37aa1dca3e20d6e1f9149446d9dfc764d7c4be59619


In [2]:
from Crypto.Hash import SHAKE256


shake = SHAKE256.new()
shake.update(b'\x12\x34')
print(shake.read(64).hex())

d003ba6b7919a4384003f645d4a25c7ee631d249edebfbdf99eeb5ee31c997edcc19a439932e29389c7be37aa1dca3e20d6e1f9149446d9dfc764d7c4be59619


In [9]:
import struct

# Constants for SHAKE256
KECCAK_ROUNDS = 24
BITRATE = 1088  # SHAKE256 uses a 1088-bit rate (136 bytes)
CAPACITY = 512  # 1600 - 1088 = 512 bits
STATE_SIZE = 1600 // 64  # Number of 64-bit words

# Keccak round constants
ROUND_CONSTANTS = [
    0x0000000000000001, 0x0000000000008082, 0x800000000000808A, 0x8000000080008000,
    0x000000000000808B, 0x0000000080000001, 0x8000000080008081, 0x8000000000008009,
    0x000000000000008A, 0x0000000000000088, 0x0000000080008009, 0x000000008000000A,
    0x000000008000808B, 0x800000000000008B, 0x8000000000008089, 0x8000000000008003,
    0x8000000000008002, 0x8000000000000080, 0x000000000000800A, 0x800000008000000A,
    0x8000000080008081, 0x8000000000008080, 0x0000000080000001, 0x8000000080008008
]

# Keccak permutation function
def keccak_f(state):
    """Applies 24 rounds of Keccak permutation."""
    for rnd in range(KECCAK_ROUNDS):
        print(f"\n--- ROUND {rnd+1} ---")

        # θ step (Theta step)
        C = [state[i] ^ state[i+5] ^ state[i+10] ^ state[i+15] ^ state[i+20] for i in range(5)]
        D = [C[(i - 1) % 5] ^ ((C[(i + 1) % 5] << 1) & 0xFFFFFFFFFFFFFFFF) for i in range(5)]
        state = [state[i] ^ D[i % 5] for i in range(25)]
        print(f"After Theta step: {state}")

        # ρ and π steps (Rho and Pi steps)
        new_state = [0] * 25
        x, y = 1, 0
        for i in range(24):
            new_state[y * 5 + x] = (state[y * 5 + x] << ((i + 1) * (i + 2) // 2)) & 0xFFFFFFFFFFFFFFFF
            x, y = y, (2 * x + 3 * y) % 5

        # χ step (Chi step)
        for y in range(5):
            row = [new_state[y * 5 + x] for x in range(5)]
            for x in range(5):
                new_state[y * 5 + x] ^= (~row[(x + 1) % 5] & row[(x + 2) % 5])

        print(f"After Chi step: {new_state}")

        # ι step (Iota step)
        new_state[0] ^= ROUND_CONSTANTS[rnd]
        print(f"After Iota step: {new_state}")
        state = new_state

    print("\nFinal State:", state)  # Debugging print
    return state

# Padding function for SHAKE256
def pad_shake256(message):
    """Pads the input message according to SHAKE256 rules."""
    padded = message + b'\x1F'  # SHAKE uses 0x1F as padding start
    pad_len = BITRATE // 8 - (len(padded) % (BITRATE // 8))  # Fill remaining space
    padded += b'\x00' * (pad_len - 1) + b'\x80'  # Append final bit
    return padded

# SHAKE256 Absorb phase
def shake256_absorb(message):
    """Absorbs input data into Keccak state."""
    state = [0] * 25  # 1600-bit state initialized to zero
    padded_msg = pad_shake256(message)

    for i in range(0, len(padded_msg), BITRATE // 8):
        block = padded_msg[i:i + BITRATE // 8]
        block_words = struct.unpack('<' + 'Q' * (BITRATE // 64), block)
        for j in range(len(block_words)):
            state[j] ^= block_words[j]  # XOR into state

        state = keccak_f(state)  # Apply Keccak permutation

    return state

# SHAKE256 Squeeze phase
def shake256_squeeze(state, output_len):
    """Extracts an arbitrary number of output bytes from the Keccak state."""
    output = b""
    while len(output) < output_len:
        for i in range(BITRATE // 64):
            output += struct.pack('<Q', state[i])  # Convert words to bytes
            if len(output) >= output_len:
                return output[:output_len]
        state = keccak_f(state)  # Apply permutation for more output
    return output

# Full SHAKE256 function
def shake256(message, output_len):
    """SHAKE256 hashing function with variable output length."""
    state = shake256_absorb(message)
    return shake256_squeeze(state, output_len)

# Example usage
message = b"\x12\x34"
output_length = 64  # 256-bit output
shake_hash = shake256(message, output_length)
print(f"SHAKE256 Hash: {shake_hash.hex()}")



--- ROUND 1 ---
After Theta step: [2044946, 2044946, 9223372036854775808, 0, 4089892, 0, 2044946, 9223372036854775808, 0, 4089892, 0, 2044946, 9223372036854775808, 0, 4089892, 0, 9223372036856820754, 9223372036854775808, 0, 4089892, 0, 2044946, 9223372036854775808, 0, 4089892]
After Chi step: [0, 4089892, 0, 0, 4089892, 0, 0, 0, 0, 0, 0, 2094024704, 0, 0, 2094024704, 0, 16609908744439988224, 0, 0, 16609908744439988224, 0, 0, 0, 0, 0]
After Iota step: [1, 4089892, 0, 0, 4089892, 0, 0, 0, 0, 0, 0, 2094024704, 0, 0, 2094024704, 0, 16609908744439988224, 0, 0, 16609908744439988224, 0, 0, 0, 0, 0]

--- ROUND 2 ---
After Theta step: [3136405298977333357, 4089893, 16609908746535968804, 14773073419362385992, 4089894, 3136405298977333356, 1, 16609908746535968804, 14773073419362385992, 2, 3136405298977333356, 2094024705, 16609908746535968804, 14773073419362385992, 2094024706, 3136405298977333356, 16609908744439988225, 16609908746535968804, 14773073419362385992, 16609908744439988226, 313640529897